In [1]:
import boto3

#get db cponnection
dynamo_db = boto3.resource('dynamodb', region_name='us-west-2')
print dynamo_db


dynamodb.ServiceResource()


In [2]:
#list all tables
client = boto3.client('dynamodb', region_name='us-west-2')
all_tables = client.list_tables()
print all_tables

{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': 'OA5T169FPGSLMAN5MUJ5AS7U4VVV4KQNSO5AEMVJF66Q9ASUAAJG'}, u'TableNames': [u'Forum-sy', u'Reply', u'Thread-sy', u'biopsy_dev', u'biopsy_dev_2', u'biopsy_dev_3', u'biopsy_dev_4', u'no_template_sample_control', u'patient_dev', u'positive_sample_control', u'sy_patient_development', u'sy_patient_event_development', u'sy_patient_history_development', u'sy_patient_history_test', u'sy_patient_test', u'ta_basic_treatment_arm_development', u'ta_basic_treatment_arm_test', u'ta_treatment_arm_development', u'ta_treatment_arm_patient_development', u'ta_treatment_arm_test']}


In [15]:
import json

sample_control_table = dynamo_db.Table('positive_sample_control')
print sample_control_table

response = sample_control_table.scan()
items = response.get('Items')


print items
    


dynamodb.Table(name='positive_sample_control')
[{u'vcf_path': u'ped-match-sample-control/MoCha/SampleControl_MoCha_3_v2_SampleControl_MoCha_3_RNA_v2.vcf', u'nextGenerationSequence': {u'passed': False, u'positiveControls': [{u'hasMatchingVariant': False, u'positiveControl': {u'function': None, u'geneName': u'PIK3CA', u'dna': u'c.333G>C', u'reference': u'G', u'variantType': u'SNV', u'purpose': None, u'position': u'178916946', u'alternative': u'C', u'identifier': u'COSM12580', u'protein': u'p.Lys111Asn', u'chromosome': u'chr3'}}, {u'hasMatchingVariant': False, u'positiveControl': {u'function': u'missense', u'geneName': u'BRAF', u'dna': u'c.1799T>A', u'reference': u'A', u'variantType': u'SNV', u'purpose': u'Substitution', u'position': u'140453136', u'alternative': u'T', u'identifier': u'COSM476', u'protein': u'p.V600E', u'chromosome': u'chr7'}}, {u'hasMatchingVariant': False, u'positiveControl': {u'function': None, u'geneName': u'BRCA2', u'dna': u'c.9281C>A', u'reference': u'C', u'variantT

In [27]:
from boto3.dynamodb.conditions import Key
import json
import decimal

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)

site = 'MoCha'
response = sample_control_table.query(
            KeyConditionExpression=Key('site').eq('MoCha')
        )
print 'total item count:', response[u'Count']



total item count: 3


In [28]:
for i in response[u'Items']:
    print 'item_molecular_id:', i['molecular_id']
    #print(json.dumps(i, cls=DecimalEncoder))

item_molecular_id: MoCha_3
item_molecular_id: MoCha_4
item_molecular_id: MoCha_5


In [33]:
response = sample_control_table.query(
            KeyConditionExpression=Key('site').eq('MoCha'), ScanIndexForward=False, Limit=1
        )
print 'total item count:', response[u'Count']
for i in response[u'Items']:
    print 'item_molecular_id:', i['molecular_id']

total item count: 1
item_molecular_id: SampleControl_MoCha_5


In [35]:
latest_molecular_id = 'SampleControl_MoCha_5'
new_molecular_id = '_'.join(latest_molecular_id.split('_')[:-1])+'_'+str(int(latest_molecular_id.split('_')[-1])+1)
print new_molecular_id

SampleControl_MoCha_6


In [36]:
response = sample_control_table.query(
            KeyConditionExpression=Key('site').eq('MoCha'), ScanIndexForward=False, Limit=1
        )
print response

{u'Count': 1, u'Items': [{u'site': u'MoCha', u'site_ip_address': u'129.43.127.133', u'molecular_id': u'SampleControl_MoCha_5', u'date_molecular_id_created': u'2016-08-11 19:23:35-0500'}], u'LastEvaluatedKey': {u'date_molecular_id_created': u'2016-08-11 19:23:35-0500', u'site': u'MoCha'}, u'ScannedCount': 1, 'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '0TP60FVNU3IAOEQTPLRDSQKJC7VV4KQNSO5AEMVJF66Q9ASUAAJG'}}


In [38]:
#validate molecualr id existense
molecular_id = 'SampleControl_MoCha_5'
site = molecular_id.split('_')[1]
response = sample_control_table.query(
            KeyConditionExpression=Key('site').eq(site)
        )
ids = []
for i in response[u'Items']:
    ids.append(i['molecular_id'])
print ids
print 'molecular id exist?'
print molecular_id in ids

[u'SampleControl_MoCha_3', u'SampleControl_MoCha_4', u'SampleControl_MoCha_5']
molecular id exist?
True
